In [14]:
# 17_search.ipynb

%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/app')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from psql_helpers import read_table_async

In [16]:
from psql_models import *

In [17]:
from psql_access import get_meeting_token
from datetime import datetime
from indexing.redis_keys import RedisKeys
from indexing.worker import IndexingWorker
from redis import Redis


In [18]:
from sqlalchemy import update
from psql_helpers import get_session

async def reset_content_indexing():
    async with get_session() as session:
        await session.execute(
            update(Content).values(is_indexed=False)
        )
        await session.commit()

# Execute
#await reset_content_indexing()

In [27]:
df = await read_table_async(User)

In [28]:
df

,id,email,username,first_name,last_name,image,created_timestamp,updated_timestamp,is_indexed
0,9717647e-fb33-4997-98af-a8b4e30f609d,test_8c25cdc5bd@example.com,None,None,None,None,2025-02-27 17:20:38.745678+00:00,2025-02-27 17:20:38.745681+00:00,False


In [38]:
await read_table_async(TranscriptAccess)

,id,transcript_id,user_id,access_level,granted_at,granted_by
0,1,662f8d0f-4640-40e8-b4da-755701726df6,9717647e-fb33-4997-98af-a8b4e30f609d,owner,2025-02-27 17:23:54.171626+00:00,9717647e-fb33-4997-98af-a8b4e30f609d
1,2,3625fc59-66ae-4e77-b67b-2fffc3a9f6a8,9717647e-fb33-4997-98af-a8b4e30f609d,owner,2025-02-27 17:23:54.171629+00:00,9717647e-fb33-4997-98af-a8b4e30f609d
2,3,490369f8-dcde-4b5c-8478-c3ab49746764,9717647e-fb33-4997-98af-a8b4e30f609d,owner,2025-02-27 17:23:54.171630+00:00,9717647e-fb33-4997-98af-a8b4e30f609d
3,4,78fad734-b23f-47df-a954-591831f9c190,9717647e-fb33-4997-98af-a8b4e30f609d,owner,2025-02-27 17:23:54.171631+00:00,9717647e-fb33-4997-98af-a8b4e30f609d
4,5,4790fd5c-0471-431a-97f8-8a7fd21e4eb6,9717647e-fb33-4997-98af-a8b4e30f609d,owner,2025-02-27 17:23:54.171631+00:00,9717647e-fb33-4997-98af-a8b4e30f609d
...,...,...,...,...,...,...
141,142,8f93b978-671c-4655-b8bd-fce94bb6e1f9,9717647e-fb33-4997-98af-a8b4e30f609d,owner,2025-02-27 17:38:01.276657+00:00,9717647e-fb33-4997-98af-a8b4e30f609d
142,143,c4acaa30-373a-474d-bb13-62c9bfecece5,9717647e-fb33-4997-98af-a8b4e30f609d,owner,2025-02-27 17:38:01.289113+00:00,9717647e-fb33-4997-98af-a8b4e30f609d
143,144,f77937a6-7631-43d5-a8c0-02de89660cca,9717647e-fb33-4997-98af-a8b4e30f609d,owner,2025-02-27 17:38:01.301467+00:00,9717647e-fb33-4997-98af-a8b4e30f609d
144,145,4981c17f-84de-4d53-86f8-6be7de03ece1,9717647e-fb33-4997-98af-a8b4e30f609d,owner,2025-02-27 17:38:01.313689+00:00,9717647e-fb33-4997-98af-a8b4e30f609d


In [36]:
df = await read_table_async(Transcript)
df.sort_values(by='start_timestamp')

,id,content_id,text_content,start_timestamp,end_timestamp,confidence,word_timing_data,segment_metadata
141,df212442-703b-4b3c-9b50-655d4e488556,4cc8ca66-6df8-5d79-b158-22552547e62c,я думал ты что-то про ну типа как хранить там ...,2025-02-13 15:32:16.430000+00:00,2025-02-13 15:32:30.750000+00:00,0.488827,"{'words': [{'word': 'я', 'start': 0.43, 'end':...","{'speaker': 'Sergey Ryabenko', 'present_user_i..."
34,e39cce44-25ea-4681-9953-372e5b1beaba,4cc8ca66-6df8-5d79-b158-22552547e62c,я думал ты что-то про ну типа как хранить там ...,2025-02-13 15:32:24.430000+00:00,2025-02-13 15:32:38.750000+00:00,0.349162,"{'words': [{'word': 'я', 'start': 0.43, 'end':...","{'speaker': 'Dmitriy Grankin', 'present_user_i..."
82,3e9b6733-eb5d-4a2c-858f-6d13ad4cedbc,4cc8ca66-6df8-5d79-b158-22552547e62c,я думал ты что-то про ну типа как хранить там ...,2025-02-13 15:32:24.430000+00:00,2025-02-13 15:32:38.750000+00:00,0.349162,"{'words': [{'word': 'я', 'start': 0.43, 'end':...","{'speaker': 'Dmitriy Grankin', 'present_user_i..."
59,882799d2-fa7b-4c02-800d-18a7d6b26e93,4cc8ca66-6df8-5d79-b158-22552547e62c,я думал ты что-то про ну типа как хранить там ...,2025-02-13 15:32:24.430000+00:00,2025-02-13 15:32:38.750000+00:00,0.349162,"{'words': [{'word': 'я', 'start': 0.43, 'end':...","{'speaker': 'Dmitriy Grankin', 'present_user_i..."
107,0d7ed6b9-0906-4b1f-9a40-d253a9ba7f1d,4cc8ca66-6df8-5d79-b158-22552547e62c,я думал ты что-то про ну типа как хранить там ...,2025-02-13 15:32:24.430000+00:00,2025-02-13 15:32:38.750000+00:00,0.349162,"{'words': [{'word': 'я', 'start': 0.43, 'end':...","{'speaker': 'Dmitriy Grankin', 'present_user_i..."
...,...,...,...,...,...,...,...,...
137,e04ebebf-e11b-4368-b884-b463652cf6f3,4cc8ca66-6df8-5d79-b158-22552547e62c,так да ну можем и технически чего-нибудь обсуд...,2025-02-13 15:44:24.170000+00:00,2025-02-13 15:44:33.070000+00:00,0.000000,"{'words': [{'word': 'так', 'start': 26.17, 'en...","{'speaker': None, 'present_user_ids': ['971764..."
125,295dd1c6-38ea-44dc-bade-2947171258dd,4cc8ca66-6df8-5d79-b158-22552547e62c,без разницы можно просто говорить ясно вопроса,2025-02-13 15:44:33.070000+00:00,2025-02-13 15:44:41.110000+00:00,0.000000,"{'words': [{'word': 'без', 'start': 35.07, 'en...","{'speaker': None, 'present_user_ids': ['971764..."
131,98817f66-af0d-4cd3-ade3-9dcda3a756f2,4cc8ca66-6df8-5d79-b158-22552547e62c,вопрос,2025-02-13 15:44:41.110000+00:00,2025-02-13 15:44:41.530000+00:00,0.000000,"{'words': [{'word': 'вопрос', 'start': 43.11, ...","{'speaker': None, 'present_user_ids': ['971764..."
139,e3095429-f55e-48f5-b89a-4b4a91dd559d,4cc8ca66-6df8-5d79-b158-22552547e62c,чего просто сейчас надо догнать пока вот эту в...,2025-02-13 15:44:41.530000+00:00,2025-02-13 15:44:50.230000+00:00,0.000000,"{'words': [{'word': 'чего', 'start': 43.53, 'e...","{'speaker': None, 'present_user_ids': ['971764..."


In [29]:
df = await read_table_async(Content)

In [30]:
df

,id,type,text,timestamp,external_id,external_id_type,last_update,parent_id,is_indexed,content_metadata
0,4cc8ca66-6df8-5d79-b158-22552547e62c,meeting,,2025-02-27 17:22:17.991935+00:00,the-zdjv-byg,google_meet,2025-02-27 17:22:17.991941+00:00,None,False,None


In [12]:
df[df['is_indexed']]

KeyError: 'is_indexed'

In [19]:
df = await read_table_async(Content)

In [20]:
df

""


In [8]:
df['is_indexed'].value_counts()


is_indexed
False    3683
Name: count, dtype: int64

In [9]:
df['type'].value_counts()


type
meeting    3683
Name: count, dtype: int64

In [30]:
processor = ContentProcessor(
            qdrant_engine=self.qdrant_engine,
            es_engine=self.es_engine
        )

NameError: name 'self' is not defined